In [ ]:
import pandas as pd

df1 = pd.read_csv("all_paper_Scores.csv", index_col=None)

In [ ]:
df1.head()

In [ ]:
df1.columns

In [ ]:
df1[["Keep-F1", "Add-F1", "Del-F1", "P","R", "F", "LevDist (Src)", "LevDist (Tgt)"]].corr()

In [ ]:
models = ["Grade Control", "Avg-Grade", "CP-Single", "Reference", "CP-Multi", "Corpus-level"]

In [ ]:
# SARI is at corpus level

for model in ["Grade Control", "CP-Multi"]:
    print(model)
    print("All", df1[(df1.model==model)][["F", "ARI Accuracy"]].astype(float).mean())
    print("No exact Match", df1[(df1.model==model) & (df1.Complex != df1.Simple)][["F", "ARI Accuracy"]].astype(float).mean())
    print("\n")

In [ ]:
df1["SG_ARI"] = df1["SG_ARI"].astype(int)

In [ ]:
all_instances = {}
for group_name, group_df in df1.groupby(["Complex", "SG"]):
    if group_name[1]==12:
        target_grades = group_df[group_df.SG_ARI==12].TG.to_list()
        if set([5]) <= set(target_grades):
            all_instances[group_name[0]] = group_df[(group_df.TG.isin([5])) & (group_df.model.isin(["Reference", "Avg-Grade", "CP-Multi"]))]

In [ ]:
len(all_instances)

In [ ]:
import random

In [ ]:
key = random.sample(list(all_instances), 1)[0]
print(key)

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
all_instances[key][["model", "SG","TG", "Simple", "SARI"]]

In [ ]:
from pingouin import ttest

for col in  ["SARI", "F", "ARI Accuracy"]:
    print(df1[df1.model=="Avg-Grade"][col].mean())
    print(df1[df1.model=="CP-Multi"][col].mean())
    
    print(col, ttest(df1[df1.model=="Avg-Grade"][col], df1[df1.model=="CP-Multi"][col], correction = False))

In [ ]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi']= 300

In [ ]:
for col in [ "Add-F1", "Keep-F1", "Del-P"]:
    g = sns.barplot(x="TG", y=col, data=df1[~(df1.model.isin(["Reference", "CP-Single"]))], hue="model")
    if col == "Keep-F1":
        plt.legend(loc='upper left', ncol=1)
    else:
        plt.legend(loc='upper right', ncol=1)
    # plt.legend([],[], frameon=False)
    plt.xlabel('Target Grade')
    plt.ylabel(col)
    plt.show()

In [ ]:
for col in [ "Add-F1", "Keep-F1", "Del-F1"]:
    g = sns.barplot(x="TG", y=col, data=df1[~(df1.model.isin(["Reference", "CP-Single", "Corpus-level"]))], hue="model")
    plt.legend(loc='upper right', ncol=1)
    # plt.legend([],[], frameon=False)
    plt.xlabel('Grade Diff')
    plt.ylabel(col)
    plt.show()

# Control Analysis

In [ ]:

complex_text_grade = read_file(f"{NEWSELA_GRADE_PATH}/newsela-grade-true/newsela-grade-true.{split}.complex")
CONTROL_FILES = {
    "Corpus-level": "../experiments/newsela-grade.test.corpus-best",
    "Avg-Grade": "newsela-grade-avg-baseline-sg-test.txt"  ,
    "CP-multi":"../controlPredictor/experiments/exp-newsela-catboost-sg-multi/output_ari.txt",
    "Oracle" :"../resources/processed_data/b6e484f0eec4c8c7bccb24a5d0cbe432/newsela-grade/newsela-grade.test.complex",
}
                

all_tokens = []
for key, out_f in CONTROL_FILES.items():
    control_out = read_file(out_f)
    for i in range(len(control_out)):
        sg, tg = [int(a.split("_")[1]) for a in complex_text_grade[i].split(" ")[:2]]
        tokens = {a.split("_")[0]: float(a.split("_")[1]) for a in control_out[i].split(" ")[:5]}
        for tok_name, tok_val in tokens.items():
            all_tokens.append([key,sg, tg,tok_name, tok_val])

In [ ]:
import pandas as pd

control_df = pd.DataFrame(all_tokens)
control_df.columns = ["Model", "SG", "TG", "Control Type", "Control Value"]

In [ ]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi']= 300

In [ ]:
sns.boxplot(data=control_df, x="Control Type", y="Control Value", hue="Model")
plt.legend(loc='upper right')

In [ ]:
fig, axes = plt.subplots(ncols=4, nrows=2, figsize=(20,15), dpi=200)
grades = list(range(2, 10))
for i, ax in zip(range(8), axes.flat):
    sns.boxplot(data=control_df[control_df.TG==grades[i]], x="Control Type", y="Control Value", hue="Model", ax=ax)
#     ax.get_legend().remove()
    ax.set_title("Target Grade %d" %grades[i])

# add legend
# handles, labels = ax.get_legend_handles_labels()
# fig.legend(handles, labels, loc='lower left')

plt.show()